In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("Marvel")
sc = SparkContext(conf = conf)

In [2]:
def countCoOccurences(line):
    elements = line.split()
    return (int(elements[0]), len(elements) - 1) # element[0] gives the superhero name, len(element) gives number of heroes

def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8")) # encoding as utf8

names = sc.textFile("file:///SparkCourse/marvel superheroes/marvel-names.txt")
namesRdd = names.map(parseNames)

lines = sc.textFile("file:///SparkCourse/marvel superheroes/marvel-graph.txt")

pairings = lines.map(countCoOccurences)
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))

mostPopular = flipped.max() # max is performed on the key which is the number of occurance and gives its name, which is the id 
                            # of he super hero

mostPopularName = namesRdd.lookup(mostPopular[1])[0] # looks up most popular by the id key and returns the line, and we choose the name

print(str(mostPopularName) + " is the most popular superhero, with " + \
    str(mostPopular[0]) + " co-appearances.")


b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
